In [1]:
%install-location $cwd/swift-install
%install '.package(path: "~/git/SwiftData")' Batcher3

Installing packages:
	.package(path: "~/git/SwiftData")
		Batcher3
With SwiftPM flags: []
Working in: /tmp/tmp9yl3uiqt/swift-install
/home/sgugger/swift/usr/bin/swift-build: /home/sgugger/anaconda3/lib/libcurl.so.4: no version information available (required by /home/sgugger/swift/usr/lib/swift/linux/libFoundationNetworking.so)
swift: /swift-base/llvm-project/llvm/include/llvm/ADT/PointerUnion.h:195: T llvm::PointerUnion<swift::ProtocolDecl *, swift::ProtocolConformance *>::get() const [PTs = <swift::ProtocolDecl *, swift::ProtocolConformance *>, T = swift::ProtocolConformance *]: Assertion `is<T>() && "Invalid accessor called"' failed.
Stack dump:
0.	Program arguments: /home/sgugger/swift/usr/bin/swift -frontend -c /home/sgugger/git/SwiftData/Batcher3/Backend.swift -primary-file /home/sgugger/git/SwiftData/Batcher3/Batcher.swift /home/sgugger/git/SwiftData/Batcher3/Collatable.swift -emit-module-path /home/sgugger/git/SwiftData/nbs/swift-install/package/.build/x86_64-unknown-linux-gnu/

: 

In [2]:
import TensorFlow
import Batcher3

In [3]:
// Base use
// Some raw items (for instance filenames)
let rawItems = 0..<512
// A heavy-compute function lazily mapped on it (for instance, opening the images)
let dataSet = rawItems.lazy.map { _ in Tensor<Float>(randomNormal: [224, 224, 3]) }
// A `Batcher` defined on this:
var batcher = Batcher(on: dataSet, batchSize: 64)
// Iteration over it (for instance, on epoch of the training loop) 
batcher.reorder(shuffled: true)
for batch in batcher {
    print(batch.shape)
}

[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]
[64, 224, 224, 3]


In [35]:
// Use with padding
// Let's create an array of things of various lengths (for instance texts)
var dataSet: [Tensor<Int32>] = []
for _ in 0..<512 {
    dataSet.append(Tensor<Int32>(
        randomUniform: [Int.random(in: 1...200)], 
        lowerBound: Tensor<Int32>(0), 
        upperBound: Tensor<Int32>(100)
    ))
}

// We need to pad those tensors to make them all the same length.
// We could do this in one lazy transform applied beforehand and pad everything
// to the same length, but it's not memory-efficient: some batches might need less
// padding. So we need to add the padding after having selected the samples we
// are trying to batch.
public struct BasicPaddingTemplate<SourceDataSet: RandomAccessCollection>: BatcherTemplate 
where SourceDataSet.Element == Tensor<Int32> {
    public func padSamples(samples: [Tensor<Int32>]) -> [Tensor<Int32>] {
        let maxLength = samples.map{ $0.shape[0] }.max()!
        return samples.map { (t: Tensor<Int32>) -> Tensor<Int32> in 
            let remaining = Tensor<Int32>(zeros: [maxLength - t.shape[0]])
            return Tensor<Int32>(concatenating: [t, remaining])
        }
    }
}

var batcher = Batcher(with: BasicPaddingTemplate(), on: dataSet, batchSize: 64)
batcher.reorder()
for b in batcher {
    print(b.shape)
}

[64, 200]
[64, 197]
[64, 200]
[64, 199]
[64, 199]
[64, 192]
[64, 199]
[64, 198]


In [37]:
// Use with a sampler
// In our previous example, another way to be memory efficient is to batch
// samples of roughly the same lengths.
public struct AdvancedPaddingTemplate<SourceDataSet: RandomAccessCollection>: BatcherTemplate 
where SourceDataSet.Element == Tensor<Int32> {
    public func padSamples(samples: [Tensor<Int32>]) -> [Tensor<Int32>] {
        let maxLength = samples.map{ $0.shape[0] }.max()!
        return samples.map { (t: Tensor<Int32>) -> Tensor<Int32> in 
            let remaining = Tensor<Int32>(zeros: [maxLength - t.shape[0]])
            return Tensor<Int32>(concatenating: [t, remaining])
        }
    }
    
    public func sampleIndices(on dataset: [Tensor<Int32>]) -> [Int] {
        return Array(0..<dataset.count).sorted { dataset[$0].shape[0] > dataset[$1].shape[0] }
    }
}

// Just giving a quick example, but there should be a separate function to shuffle a bit 
// the sorted thing
var batcher = Batcher<AdvancedPaddingTemplate<[Tensor<Int32>]>, [Tensor<Int32>]>(with: AdvancedPaddingTemplate(), on: dataSet, batchSize: 64)
batcher.reorder()
for b in batcher {
    print(b.shape)
}

[64, 200]
[64, 173]
[64, 147]
[64, 124]
[64, 96]
[64, 71]
[64, 46]
[64, 23]


In [38]:
// Sometimes the shuffle method needs to be applied on the dataset itself, 
// like for language modeling. Here is a base version of that to test
// the API allows it.
struct DataSet: RandomAccessCollection {
    typealias Index = Int
    typealias Element = Tensor<Int32>
    
    let numbers: [[Int]]
    let sequenceLength: Int
    // The texts all concatenated together
    private var stream: [Int]
    
    var startIndex: Int { return 0 }
    var endIndex: Int { return stream.count / sequenceLength }
    func index(after i: Int) -> Int { i+1 }
    
    init(numbers: [[Int]], sequenceLength: Int) {
        self.numbers = numbers
        self.sequenceLength = sequenceLength
        stream = numbers.reduce([], +)
    }
    
    subscript(index: Int) -> Tensor<Int32> {
        get { 
            let i = index * sequenceLength
            return Tensor<Int32>(stream[i..<i+sequenceLength].map { Int32($0)} )
        }
    }
    
    mutating func shuffle() {
        stream = numbers.shuffled().reduce([], +)
    }
}

In [39]:
//Let's create such a DataSet
let numbers: [[Int]] = [[1,2,3,4,5], [6,7,8], [9,10,11,12,13,14,15], [16,17,18]]
let dataset = DataSet(numbers: numbers, sequenceLength: 3)

In [44]:
public struct ShufflingTemplate: BatcherTemplate {
    public typealias SourceDataSet = DataSet 
    public func shuffleIndices(on dataset: inout DataSet, indices: [Int]) -> [Int] {
        dataset.shuffle()
        return indices
    }
}

In [45]:
//Now let's look at what it gives us:
var batcher = Batcher(with: ShufflingTemplate(), on: dataset, batchSize: 3)
batcher.reorder()
for b in batcher {
    print(b)
}

[[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]]
[[10, 11, 12],
 [13, 14, 15],
 [16, 17, 18]]


In [46]:
batcher.reorder(shuffled: true)
for b in batcher {
    print(b)
}

[[ 9, 10, 11],
 [12, 13, 14],
 [15,  6,  7]]
[[ 8, 16, 17],
 [18,  1,  2],
 [ 3,  4,  5]]
